# Homework: Phrasal Chunking

This is your documentation for the chunker homework.

In [13]:
import perc
import default
import sys
from collections import defaultdict

feat_vec = {}
tagset = []
train_data = []

def weight_change(feat_vec, truth, predicted, feat_list):
    index = 0
    for i, feature in enumerate(feat_list):
        if i != 0 and feat_list[i][0:3] == 'U00':
            index += 1
        gold_key = (feature, truth[index])
        if gold_key in feat_vec:
            feat_vec[gold_key] += 1
        else:
            feat_vec[gold_key] = 1
        
        bad_key = (feature, predicted[index])
        if bad_key in feat_vec:
            feat_vec[bad_key] -= 1
        
        if feat_vec[bad_key] == 0:
            feat_vec.pop(bad_key, None)
        

def perc_train(train_data, tagset, numepochs):
    feat_vec = defaultdict(int)
    N = len(train_data)
    for run in range(numepochs):
        for training_number in range(N):
            predicted = perc.perc_test(feat_vec, train_data[training_number][0], train_data[training_number][1], tagset, default_tag='B-NP')
            truth = [tags.split()[2] for tags in train_data[training_number][0]]
            weight_change(feat_vec, truth, predicted, train_data[training_number][1])
    return feat_vec

tagset = perc.read_tagset("data/tagset.txt")
print("reading data ...", file=sys.stderr)
train_data = perc.read_labeled_data("data/train.txt.gz", "data/train.feats.gz", verbose=False)
print("done.", file=sys.stderr)
feat_vec = perc_train(train_data, tagset, 10)
perc.perc_write_to_file(feat_vec, "default.model")
print("wrote model to disk")

reading data ...
done.


wrote model to disk


In [14]:
%%capture --no-stderr output
print("reading test data ...", file=sys.stderr)
test_data = perc.read_labeled_data("data/dev.txt", "data/dev.feats", verbose=False)
print("done.", file=sys.stderr)
feat_vec = perc.perc_read_from_file("default.model")
perc.perc_testall(feat_vec, test_data, tagset)

reading test data ...
done.


In [15]:
import score_chunks
boundary = "-X-" # something to use as boundary between sentences
outside = "O" # tag used to mark the outside of any chunk
conlleval = False # use conlleval (should be False for most use cases)
numfeats = 2 # number of columns to consider as features, typically "word POStag"
(test, _) = score_chunks.readTestFile(str(output), boundary, outside, conlleval, numfeats)
with open("data/reference500.txt") as f:
    (reference, _) = score_chunks.readTestFile(f.read(), boundary, outside, conlleval, numfeats)
print("Score: %.2f" % score_chunks.corpus_fmeasure(reference, test, False))

processed 500 sentences with 10375 tokens and 5783 phrases; found phrases: 5836; correct phrases: 5274
             ADJP: precision:  59.29%; recall:  67.68%; F1:  63.21; found:    113; correct:     99
             ADVP: precision:  69.86%; recall:  72.28%; F1:  71.05; found:    209; correct:    202
            CONJP: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      5; correct:      5
             INTJ: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:      1
               NP: precision:  91.20%; recall:  91.84%; F1:  91.52; found:   3047; correct:   3026
               PP: precision:  96.24%; recall:  96.31%; F1:  96.28; found:   1222; correct:   1221
              PRT: precision:  80.00%; recall:  54.55%; F1:  64.86; found:     15; correct:     22
             SBAR: precision:  71.77%; recall:  83.18%; F1:  77.06; found:    124; correct:    107
               VP: precision:  91.28%; recall:  91.36%; F1:  91.32; found:   1101; correct:   1100
accura